# 1. Imports

In [28]:
import numpy as np
import pandas as pd 
import plotly.express as px
import datetime as dt
import plotly.graph_objects as go
import re

# 2. Load data

In [2]:
df = pd.read_csv("../data/processed/online_retail_II_2009.csv")

In [3]:
display(df.head(3))
display(df.info())

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45228 entries, 0 to 45227
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      45228 non-null  object 
 1   StockCode    45228 non-null  object 
 2   Description  45000 non-null  object 
 3   Quantity     45228 non-null  int64  
 4   InvoiceDate  45228 non-null  object 
 5   Price        45228 non-null  float64
 6   Customer ID  31760 non-null  float64
 7   Country      45228 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 2.8+ MB


None

# 3. Clean data

In [4]:
# Print amount of missing values
print(df.isnull().sum().sort_values(ascending = False))

Customer ID    13468
Description      228
Invoice            0
StockCode          0
Quantity           0
InvoiceDate        0
Price              0
Country            0
dtype: int64


In [5]:
# Remove missing values
df.dropna(inplace=True)

In [6]:
# Summary statistics
df.describe([0.01, 0.05, 0.10, 0.20, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,20%,50%,90%,95%,99%,max
Quantity,31760.0,12.288602,63.608944,-600.0,-6.00,1.00,1.00,1.00,4.0,24.00,36.0,144.00,5184.00
Price,31760.0,3.354071,15.432042,0.0,0.21,0.42,0.65,1.25,2.1,6.75,8.5,14.95,1998.49
Customer ID,31760.0,15464.516908,1664.830594,12346.0,12455.00,12835.00,13085.00,13795.60,15503.0,17841.00,17913.0,18181.00,18287.00


Negative values are refunds, refunded invoices contain C in the invoice ID.

In [7]:
# Remove negative values for invoices
df = df[~df["Invoice"].str.contains("C", na = False)]

df.describe([0.01, 0.05, 0.10, 0.20, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,20%,50%,90%,95%,99%,max
Quantity,30761.0,13.010013,64.188578,1.0,1.00,1.00,1.00,1.00,4.0,24.00,40.0,144.00,5184.00
Price,30761.0,3.195946,9.055711,0.0,0.21,0.42,0.65,1.25,2.1,6.75,8.5,12.75,1213.02
Customer ID,30761.0,15479.710055,1664.737204,12346.0,12464.60,12835.00,13087.00,13813.00,15518.0,17841.00,17920.0,18181.00,18286.00


In [8]:
# Remove where price is 0
df = df[df["Price"] > 0]

In [9]:
# Change country EIRE to Ireland
df.loc[df["Country"] == "EIRE", "Country"] = "Ireland"

In [10]:
# Convert InvoiceDate to datetime
df.loc[:, 'InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [11]:
# Calculate revenue 2 decimal places
df.loc[:, 'Revenue'] = df['Quantity'] * df['Price']
df.loc[:, 'Revenue'] = df['Revenue'].round(2)

# 

# 4. EDA

### 4.1 Distribution of Quantity & Price

In [12]:
# Boxplot
fig = go.Figure()

# Add box traces for Quantity and Price
for col in ['Quantity', 'Price']:
    fig.add_trace(go.Box(
        x=df[col],
        name=col,
        orientation='h'
    ))

# Update layout
fig.update_layout(
    width=1000,
    title='Boxplot of Quantity and Price',
    xaxis_title='Value (log scale)',
    xaxis_type='log'
)

fig.show()

### 4.2 Top-N analysis 

In [13]:
# Barchart
country_counts = df['Country'].value_counts().head(3)
fig = go.Figure(data=[
    go.Bar(
        x=country_counts.index,
        y=country_counts.values
    )
])

# Update layout
fig.update_layout(
    width=500,
    title='Top 3 Countries by Number of Transactions',
    xaxis_title='Country',
    yaxis_title='Count'
)

fig.show()

In [14]:
# Barchart top-selling products
# Get top 3 products and their counts
product_counts = df['StockCode'].value_counts().head(3)

# Get descriptions for top products
top_products = pd.DataFrame({
    'StockCode': product_counts.index,
    'Count': product_counts.values
})
top_products = top_products.merge(df[['StockCode', 'Description']].drop_duplicates(), on='StockCode')

fig = go.Figure(data=[
    go.Bar(
        x=top_products['StockCode'],
        y=top_products['Count'],
        hovertext=top_products['Description'],
        hoverinfo='text+y'
    )
])

# Update layout
fig.update_layout(
    width=500,
    title='Top 3 Products by Number of Transactions<br><sup>Hover over bars to see product descriptions</sup>',
    xaxis_title='Stock Code',
    yaxis_title='Count'
)

fig.show()

# 5. Customer Segmentation - RFM analysis

In [15]:
# Create snapshot date (most recent transaction date + 1 day)
snapshot_date = df['InvoiceDate'].max() + pd.DateOffset(days=1)

In [16]:
# Aggregate data by Customer ID
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'Invoice': 'nunique',                                      # Frequency
    'Revenue': 'sum'                                           # Monetary
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

In [21]:
# Assign quartile-based scores (1-4)
rfm['RecencyScore'] = pd.qcut(rfm['Recency'], q=5, labels=[5, 4 , 3, 2, 1])
rfm['FrequencyScore'] = pd.qcut(rfm['Frequency'].rank(method="first"), q=5, labels=[1,2,3,4,5])
rfm['MonetaryScore'] = pd.qcut(rfm['Monetary'], q=5, labels=[1, 2, 3, 4, 5])

# Combine scores
rfm['RFM_Score'] = (
    rfm['RecencyScore'].astype(str) + 
    rfm['FrequencyScore'].astype(str) + 
    rfm['MonetaryScore'].astype(str)
)

rfm.head(3)

,CustomerID,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score
0,12346.0,6,5,113.50,5,5,1,551
1,12358.0,16,1,1429.83,2,1,5,215
2,12359.0,8,2,838.89,4,4,5,445


In [25]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk', 
    r'[1-2]5': "Can't Lose",
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [29]:
# First ensure RFM scores are strings
rfm['RFM_Score'] = rfm['RFM_Score'].astype(str)

# Extract first two digits for segmentation (Recency + Frequency)
rfm['RF_Frequency_Recency'] = rfm['RFM_Score'].str[:2]

# Create function to map segments using regex
def assign_segment(rf_code):
    for pattern, segment in seg_map.items():
        if re.match(pattern, rf_code):
            return segment
    return 'Other'  # Fallback category

# Apply segmentation
rfm['Segment'] = rfm['RF_Frequency_Recency'].apply(assign_segment)

# Sort segments by business priority
segment_order = [
    'Champions', 'Loyal Customers', 'Potential Loyalists',
    'New Customers', 'Promising', 'Need Attention',
    'About to Sleep', 'At Risk', "Can't Loose", 
    'Hibernating'
]


In [33]:
# Visualization 1: Interactive Treemap
fig = px.treemap(rfm.groupby('Segment', observed=False).size().reset_index(name='Count'),
                 path=['Segment'], values='Count',
                 title='Customer Segmentation Distribution | RFM Analysis',
                 width=800,
                 height=800)
fig.update_traces(textinfo="label+value+percent parent")
fig.show()

In [44]:
# Get high-value customers needing attention
high_value_risk = rfm[
    (rfm['Segment'].isin(['Need Attention', 'At Risk'])) & 
    (rfm['Monetary'] > rfm['Monetary'].quantile(0.75))
]

print(f"High-value customers needing attention: {len(high_value_risk)}")
print(high_value_risk[['CustomerID', 'Recency', 'Frequency', 'Monetary', 'Segment']].head())

High-value customers needing attention: 23
     CustomerID  Recency  Frequency  Monetary  Segment
53      12913.0       17          2    741.96  At Risk
59      12931.0       20          2   1268.75  At Risk
162     13526.0       23          2   1182.00  At Risk
234     13958.0       19          2    811.55  At Risk
335     14548.0       18          2    680.28  At Risk


In [45]:
rfmStats = rfm[["Segment","Recency","Frequency", "Monetary"]].groupby("Segment").agg(["mean","median","count", "std"])
rfmStats.columns = rfmStats.columns.map('_'.join).str.strip('|')
rfmStats

,Recency_mean,Recency_median,Recency_count,Recency_std,Frequency_mean,Frequency_median,Frequency_count,Frequency_std,Monetary_mean,Monetary_median,Monetary_count,Monetary_std
Segment,,,,,,,,,,,,
About to Sleep,13.500000,14.0,90,1.515834,1.000000,1.0,90,0.000000,396.959889,316.920,90,336.887766
At Risk,19.722581,20.0,155,2.416221,1.122581,1.0,155,0.329018,359.088000,281.020,155,408.646773
Can't Lose,18.562500,18.0,16,2.475715,3.125000,2.0,16,3.242941,928.318125,413.535,16,1759.656664
Champions,4.804196,6.0,143,2.059874,3.286713,3.0,143,2.272377,1942.528322,811.960,143,4206.747690
Hibernating,19.652632,20.0,190,2.262033,1.000000,1.0,190,0.000000,378.206368,282.715,190,357.726161
Loyal Customers,10.528662,10.0,157,2.344006,2.127389,2.0,157,1.119330,943.875541,556.150,157,1470.655677
Need Attention,13.333333,14.0,54,1.504710,1.000000,1.0,54,0.000000,382.886481,296.665,54,458.749562
New Customers,5.615385,6.0,26,1.551178,1.000000,1.0,26,0.000000,461.870769,312.580,26,499.973480
Potential Loyalists,7.688889,8.0,90,1.952494,1.000000,1.0,90,0.000000,395.918556,294.060,90,492.715384
